<a href="https://colab.research.google.com/github/coder272377/dvp/blob/develop/tfew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top Five Equal Weight



In [ ]:
interval = "5m"
previous_days = 5
tickers = "AAPL TSLA GOOG AMZN MSFT".split()



## Package management



In [ ]:
import datetime
import functools
import math

from typing import List, Tuple, Union



In [ ]:
Version = Union[List[int], Tuple]



In [ ]:
def check_version(req:Version, version_str:str):
  v0, v1, v2 = version_str.split('.')

  result = False

  if int(v0) > req[0]:
    result = True
  elif int(v1) > req[1]:
    result = True
  elif int(v2) >= req[2]:
    result = True

  return result



Test function `check_version()` before using



In [ ]:
assert check_version([0, 9, 9], "0.10.0")



In [ ]:
assert check_version([0, 10, 0], "0.10.0")



How do I unload (reload) a Python module? https://stackoverflow.com/questions/437589/how-do-i-unload-reload-a-python-module



In [ ]:
import pandas as pd
if not check_version([1, 3, 5], pd.__version__):
  !pip install --upgrade pandas
  import importlib
  importlib.reload(pd)
  assert check_version([1, 3, 5], pd.__version__)



Unable to get stock data from yahoo with pandas_datareader

https://stackoverflow.com/questions/68327968/unable-to-get-stock-data-from-yahoo-with-pandas-datareader/68435757#68435757

Download Financial Dataset Using Yahoo Finance in Python | A Complete Guide https://www.analyticsvidhya.com/blog/2021/06/download-financial-dataset-using-yahoo-finance-in-python-a-complete-guide/



yfinance Library – A Complete Guide https://algotrading101.com/learn/yfinance-guide/


In [ ]:
import pandas as pd



In [ ]:
try:
  import yfinance as yf
except ModuleNotFoundError:
  !pip install yfinance
  !pip install yahoofinancials
  import yfinance as yf



Charting Candlestick_OHLC one minute bars with Pandas and Matplotlib https://stackoverflow.com/questions/41821916/charting-candlestick-ohlc-one-minute-bars-with-pandas-and-matplotlib

Candlestick Chart in Python (mplfinance, plotly, bokeh, bqplot and cufflinks) https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh#1

In [ ]:
try:
  import mplfinance as fplt
except ModuleNotFoundError:
  !pip install mplfinance
  import mplfinance as fplt



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
import scipy.signal as signal



## Dates



In [ ]:
@functools.lru_cache()
def tz_ny() -> str:
  return "America/New_York"



In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
@functools.lru_cache()
def USBday():
  return pd.tseries.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())



In [ ]:
@functools.lru_cache()
def now() -> pd.Timestamp:
  return pd.Timestamp.now(tz=tz_ny())



In [ ]:
@functools.lru_cache()
def get_start_date(n_days:int, end_date=now(), unit=USBday()):
  return end_date + unit * (-n_days)



In [ ]:
def test_get_start_date__default_argument():
  result = get_start_date(5)
  assert result < now(), (result, now)



In [ ]:
test_get_start_date__default_argument()



In [ ]:
@functools.lru_cache()
def get_nyse_open(t:pd.Timestamp):
  return pd.Timestamp(
    year=t.year, month=t.month, day=t.day, hour=9, minute=30, tz=tz_ny(),
  )



## Data-getters



In [ ]:
@functools.lru_cache()
def get_df_start_end(ticker:str, start:pd.Timestamp, end:pd.Timestamp=now(), interval:str=interval):
  assert start < end
  if interval[-1] == 'm':
    delta = now() - start
    assert (pd.Timedelta(60, 'D') > delta), f"{start} older than 60 days for interval {interval!r}"
  tic = yf.Ticker(ticker)
  return tic.history(start=start, end=end, interval=interval)



In [ ]:
def get_df(ticker:str, n_days:int, end_date:pd.Timestamp=now(), interval:str=interval):
  return get_df_start_end(ticker, get_start_date(n_days, end_date=end_date), end=end_date, interval=interval)



In [ ]:
def get_days_df(ticker:str, n_days:int=previous_days, end_date:pd.Timestamp=now(), interval:str="1D") -> pd.DataFrame:
  return get_df(ticker, n_days, end_date, interval)



In [ ]:
def get_minutes_df(ticker:str, n_days:int=2, end_date:pd.Timestamp=now(), interval:str=interval) -> pd.DataFrame:
  return get_df(ticker, n_days, end_date, interval)



Test daily data getter functions



In [ ]:
def test_get_days_df_sunday():
  tic = "GOOG"
  days = 5
  end_date = pd.Timestamp("2022-01-23 10:50")
  df_days = get_days_df(tic, n_days=days, end_date=end_date, interval="1D")

  assert pd.Timestamp("2022-01-21 00:00:00") == df_days.index[-1]



In [ ]:
test_get_days_df_sunday()



In [ ]:
def test_get_days_df__holiday():
  tic = "MSFT"
  days = 5
  end_date = pd.Timestamp("2021-06-02 09:50")
  df_days = get_days_df(tic, n_days=days, end_date=end_date, interval="1D")

  assert pd.Timestamp("2021-06-01 00:00:00") == df_days.index[-1], df_days



In [ ]:
test_get_days_df__holiday()



Test minute data getter functions



In [ ]:
def test_get_minutes_df__today():
  tic = "GOOG"
  days = 2
  end_date = now()
  df_minutes = get_minutes_df(tic, n_days=days, end_date=end_date, interval="5m")

  assert pd.Timedelta(20, 'h') < (df_minutes.index[-1] - df_minutes.index[0]), df_minutes



In [ ]:
test_get_minutes_df__today()



In [ ]:
def calc_pct(tic:str):
  df_minutes = get_minutes_df(tic)

  assert df_minutes.shape[0], df_minutes.shape

  fin = df_minutes.index[-1]
  fin_begin = get_nyse_open(fin)

  prev = df_minutes.index < fin_begin
  index_prev = df_minutes.index[prev]

  df_prev = df_minutes.loc[index_prev, :]
  assert df_prev.shape[0], df_prev.shape

  previous_close = df_prev.at[df_prev.index[-1], "Close"]

  ser_current_date = df_minutes["Close"].drop(index_prev)
  assert df_prev.index[-1] < ser_current_date.index[0]

  return ser_current_date.mul(100.0 / previous_close) - 100.0



## Chart-plotter



In [ ]:
def plot_candlestick(df:pd.DataFrame, ax=None):
  if ax is None:
    fig, ax = plt.subplots(figsize=(16, 9))

  fplt.plot(
      df,
      type='candle',
      style='charles',
      ylabel='Price ($)',
      ax=ax,
      datetime_format="%y/%m/%d %H:%M"
  );

  ax.grid(True)

  return ax



In [ ]:
tic = "TSLA"

fig, ax = plt.subplots(2, 1, figsize=(24,10))

df_minutes = get_minutes_df(tic)
df_pct = calc_pct(tic)
plot_candlestick(
    df_minutes.drop(
        df_minutes.index[df_minutes.index < get_nyse_open(df_minutes.index[-1])]
    ),
    ax=ax[0]
)

xlim = ax[0].get_xlim()
xtick = ax[0].get_xticks()

df_pct.plot.bar(ax=ax[1], grid=True, ylabel="%")
ax[1].set_xticks(xtick)
ax[1].set_xlim(xlim)

